# Overview

This notebook downloads code diffs and samples a desired number of negative commits from the RA21 dataset.



In [27]:
import re
import pandas as pd
import numpy as np
import requests
import time
import math
from datetime import datetime

In [2]:
def df_to_csv(df, path):
    print(f'Trying to save {len(df)} rows.')
    df.to_csv(path, index = False, errors='replace')

In [ ]:
def download_diff(owner, repo, hash, token=None):
    # Downloads a commit's code diff using github's API.
    # Example call: 
    #   download_diff(owner = "TQRG", repo = "security-patches-dataset", hash = "6ee8095bb232d1e190bf453af09fc87cae558d02")
    #
    # Returns a request object.
    query_url = f"https://api.github.com/repos/{owner}/{repo}/commits/{hash}"
    headers = {'Accept': 'application/vnd.github.v3.diff'}
    if token:
        headers['Authorization'] = f'token {token}'
    return requests.get(query_url, headers=headers)

# demo:
r = download_diff(owner = "TQRG", repo = "security-patches-dataset", hash = "6ee8095bb232d1e190bf453af09fc87cae558d02")
r.text

In [11]:
def download_diffs_all_positive(df, rate_limit_per_sec, token=None, report_every=100, rate_limit_leeway=0.05):
    # Downloads all code diffs for the positive dataset.
    # Code diffs are written under column "diff" of `df`.
    #
    # Inputs:
    #   `df`: Pandas dataframe of the positive sample dataset positive.csv
    #   `rate_limit_per_sec`: Github API's rate limit. (units: per second). For example
    #       5000 requests per hour == 5000/60/60 per second.
    #   `token`: Your Github API token. Optional.
    #   `report_every`: Prints an update after working thru every `report_every` rows. Default 100 rows.
    #   `rate_limit_leeway`: Margin of safety on the rate limit to avoid accidentally triggering it. Default 5%.
    #
    # Returns:
    #   - A pandas dataframe containing the code diffs column
    #   - An error List.
    #

    errors = []
    df['diff'] = np.nan # create empty column to write code diffs to.
    leeway = 1 + rate_limit_leeway
    duration_per_request_minimum_s = (1.0 / rate_limit_per_sec) * leeway

    for i in df.index:
        start = time.time()
        skip = False

        hash = df['sha'][i]
        project = df['project'][i]
        if 'github' not in project.lower():
            print(f'[WARN] {project} (row {i}) not on Github. Cannot use Github API. Skipped.')
            errors.append(f'row indexed {i} not-github-project')
            skip = True   #skip row
        
        split = project.split('/')
        owner, repo = split[-2], split[-1]
        r = download_diff(owner, repo, hash, token)
        if 200 != r.status_code:
            print(f'[WARN] GET request for {project} (row {i}) failed with status code {r.status_code}. Skipped.')
            errors.append(f'row indexed {i} status-not-200')
            skip = True   #skip row

        if not skip:
            df['diff'][i] = r.text

        if (i+1) % report_every == 0:
            print(f'[INFO] Done {i+1} rows.')

        duration = time.time() - start
        if duration < duration_per_request_minimum_s:
            sleep_s = duration_per_request_minimum_s - duration
            time.sleep(sleep_s)

    if errors:
        print('[WARN] ERRORS DETECTED. SEE VARIABLE `errors` FOR DETAILS.')
    else:
        print('[INFO] No errors.')

    return df, errors

In [ ]:
# Download all code diffs for positive dataset.

df = pd.read_csv('security-patches-dataset/dataset/positive.csv')
token = 'nfhrgioKJNIOEng358943'   # Your github token goes here.
rate_limit_per_sec = 5000 / 60 / 60     # Github API's rate limit for basic (non-enterprise) users.
df, errors = download_diffs_all_positive(df, rate_limit_per_sec=rate_limit_per_sec, token=token, report_every=100)
display(df)
df_to_csv(df, path='positive+CC.csv')

# Filter positive & negative dataset for projects that appear in both sets.

In [ ]:
# Get projects in positive set.

dfp = pd.read_csv('positive+CC.csv')
dfp = dfp.dropna(subset=['diff'])
# print(dfp)
projects = dfp['project'].str.lower().unique()
projects = [p[p.find('github.com'): ] for p in projects] # remove "https://www." part
print(projects[:10], len(projects))

# Get porjects in negative set.

dfn = pd.read_csv('security-patches-dataset/dataset/negative.csv')
projects_n = dfn['github'].str.lower()
projects_n = [p[p.find('github.com'): ] for p in projects_n] # remove "https://www." part
projects_n = [p[ : p.rfind('/commit') ] for p in projects_n] # remove "/commit/0d533c3615f7c54fa2b64d16" part
projects_n = list(set(projects_n)) # unique-fy
print(projects_n[:10], len(projects_n))

# Find overlap between positive and negative sets.

overlap = set(projects) & set(projects_n)
missing_in_n = len(set(projects) - set(projects_n))
missing_in_p = len(set(projects_n) - set(projects))
print(f'{len(overlap) = } {missing_in_n = } {missing_in_p = }')

# Will only work on projects present in both pos & neg sets.

keep_list = [re.escape(project) for project in overlap]
dfp_new = dfp[dfp['project'].str.contains('|'.join(keep_list), regex=True, flags=re.IGNORECASE)]
dfn_new = dfn[dfn['github'].str.contains('|'.join(keep_list), regex=True, flags=re.IGNORECASE)]
print(f'{len(dfp_new) = } {len(dfn_new) = }')

# Save new sets.

df_to_csv(dfp_new, path=f'positive+CC-{len(overlap)}repos.csv')
df_to_csv(dfn_new, path=f'negative-{len(overlap)}repos.csv')

# Download code changes for chosen negative samples

Two options:

1. Sample negative commits constrained to repositories present in both positive & negative data.

2. Sample negative commits without constrain.

In [ ]:
# OPTION 1: 
# Sample negative commits constrained to repositories present in both positive & negative data. 


# Set number of negative samples N to download code changes for:
dfp = pd.read_csv('positive+CC-900repos.csv')
N = len(dfp)
# N = 2 * len(dfp)   # Example: sample 2x as many.
print(f'{N = }')

# Read in negative dataset:
dfn = pd.read_csv('negative-900repos.csv')
projects_n = dfn['github'].str.lower()
projects_n = [p[p.find('github.com'): ] for p in projects_n] # remove "https://www." part
projects_n = [p[ : p.rfind('commit') ] for p in projects_n] # remove "commit/0d533c3615f7c54fa2b64d16" part
projects_n = set(projects_n) # unique-fy
print(f'{len(projects_n) = }')

# Sample negative commits.

dfn['select'] = np.nan # make empty column.
extra = 2   # extra samples in case some commits don't have code diffs available.
samples_per_project = math.ceil(N / len(projects_n)) + extra

for i, project in enumerate(projects_n):
    df = dfn[dfn['github'].str.contains(re.escape(project), regex=True, flags=re.IGNORECASE)] # get rows of this project
    if len(df) == 0:
        print(f'[WARN] {project =} has no rows.')
    seed = i
    idxs_select = df.sample(n=min(samples_per_project, len(df)), random_state=seed).index
    dfn['select'][idxs_select] = 'A'   # 'A' denotes selected.
    


In [ ]:
# OPTION 2: 
# Sample negative commits without constrain.

# Set number of negative samples N to download code changes for:
dfp = pd.read_csv('positive+CC.csv')
# N = len(dfp)
N = 2 * len(dfp)
print(f'{N = }')

# Read in negative dataset:
dfn = pd.read_csv('security-patches-dataset/dataset/negative.csv')
projects_n = dfn['github'].str.lower()
projects_n = [p[p.find('github.com'): ] for p in projects_n] # remove "https://www." part
projects_n = [p[ : p.rfind('commit') ] for p in projects_n] # remove "commit/0d533c3615f7c54fa2b64d16" part
projects_n = set(projects_n) # unique-fy
print(f'{len(projects_n) = }')

# Sample negative commits.

dfn['select'] = np.nan # make empty column.
extra = 1   # extra samples in case some commits don't have code diffs available.
samples_per_project = math.ceil(N / len(projects_n)) + extra

for i, project in enumerate(projects_n):
    df = dfn[dfn['github'].str.contains(re.escape(project), regex=True, flags=re.IGNORECASE)] # get rows of this project
    if len(df) == 0:
        print(f'[WARN] {project =} has no rows.')
    seed = i + 100
    # print(df)
    idxs_select = df.sample(n=min(samples_per_project, len(df)), random_state=seed).index
    # print(idxs_select)
    dfn['select'][idxs_select] = 'A'    # 'A' denotes selected.

In [ ]:
# View selected negative commits.

def view(dfn):
    dfn_select = dfn[dfn['select'] == 'A']
    pp = dfn_select['github'].str.lower()
    pp = [p[p.find('github.com'): ] for p in pp] # remove "https://www." part
    pp = [p[ : p.rfind('commit') ] for p in pp] # remove "commit/0d533c3615f7c54fa2b64d16" part
    pp = list(set(pp)) # unique-fy

    print(f'num projects: {len(pp)}. num rows {len(dfn_select) =}')

    display(dfn_select)

view(dfn)

In [ ]:
def download_diffs_for_negative_dataset(df, 
                        duration_per_request_minimum_s,
                        token=None, 
                        report_every=100):

    errors = []
    n_done = 0

    for i in df.index:
        start = time.time()
        skip = False
        
        url = df['github'][i]
        _,_,_,owner,repo,_,hash = url.split('/')

        if 'github' not in url.lower():
            print(f'[WARN] {url} (row indexed {i}) not on Github. Cannot use Github API. Skipped.')
            errors.append(f'row {i} not-github-project')
            skip = True  # skip row

        r = download_diff(owner, repo, hash, token)
        
        if 200 != r.status_code:
            print(f'[WARN] GET request for {url} (row indexed {i}) failed with status code {r.status_code}. Skipped.')
            errors.append(f'row indexed {i} status-not-200')
            skip = True  # skip row

        if not skip:
            df['diff'][i] = r.text

        n_done += 1
        if 0 == n_done % report_every:
            print(f'[INFO] Done {n_done} rows.')

        duration = time.time() - start
        if duration < duration_per_request_minimum_s:
            sleep_s = duration_per_request_minimum_s - duration
            time.sleep(sleep_s)

    if errors:
        print('[WARN] ERRORS DETECTED. SEE VARIABLE `errors` FOR DETAILS.')
    else:
        print('[INFO] No errors.')

    return df, errors

token = 'ajksdgfuiy89y4tbr9g984'   # your github token goes here.
rate_limit_per_sec = (5000 - 1) / 60 / 60     # Github API's rate limit for basic (non-enterprise) users is 5000 requests per hour. -1 for leeway.
duration_per_request_minimum_s = (1.0 / rate_limit_per_sec) * 1.07      # additional 7% time leeway.

dfn_select = dfn[dfn['select'] == 'A']
dfn_select['diff']  = np.nan # make empty column

df_cc, errors = download_diffs_for_negative_dataset(dfn_select, duration_per_request_minimum_s=duration_per_request_minimum_s, token=token)

display(df_cc)

# Save df
dt = '{:%Y-%m-%d-%H%M}'.format(datetime.now())
df_to_csv(df_cc, path=f'negative-{dt}.csv')   #  filename like: negative-2017-06-27-1856.csv

# If you had chosen Option 1, the saved file is conceptually similar to `negative+cc-900repos`

In [ ]:
# Truncate code diffs that are too long if you'd like.

df=pd.read_csv('negative-2021-09-10-1818.csv')
print(max(df['diff'].str.len()))
df['diff'] = df['diff'].str.slice(0, 200000)
print(max(df['diff'].str.len()))
dt = '{:%Y-%m-%d-%H%M}'.format(datetime.now())
df_to_csv(df_cc, path='negative-%s.csv'%( '{:%Y-%m-%d-%H%M}'.format(datetime.now()) ))   #  filename like: negative-2017-06-27-1856.csv